In [1]:
import sys
sys.path.append('/home/jetson/dofbot_pro/dofbot_apriltag/scripts')

In [2]:
import cv2 as cv
import threading
import random
from time import sleep
import ipywidgets as widgets
from IPython.display import display
from apriltag_identify import ApriltagIdentify
from dofbot_utils.fps import FPS
from dofbot_utils.robot_controller import Robot_Controller

In [3]:
apriltag_Identify = ApriltagIdentify()
model = 'General'


In [4]:
robot = Robot_Controller()
robot.move_look_map()
fps = FPS()

In [5]:
button_layout      = widgets.Layout(width='320px', height='60px', align_self='center')
output = widgets.Output()
# 退出 exit
exit_button = widgets.Button(description='Exit', button_style='danger', layout=button_layout)
# 图像控件 Image widget
imgbox = widgets.Image(format='jpg', height=480, width=640, layout=widgets.Layout(align_self='center'))
# 垂直布局 Vertical layout
display_box = widgets.VBox([imgbox, exit_button], layout=widgets.Layout(align_self='center'))


In [6]:
def exit_button_Callback(value):
    global model
    model = 'Exit'

exit_button.on_click(exit_button_Callback)

In [7]:
def camera():
    global HSV_learning,model
    # 打开摄像头 Open camera
    capture = cv.VideoCapture(0)
    capture.set(3, 640)
    capture.set(4, 480)
    capture.set(5, 30)
    # Be executed in loop when the camera is opened normally 
    # 当摄像头正常打开的情况下循环执行
    while capture.isOpened():
        try:
            _, img = capture.read()
            fps.update_fps()
            img, msg = apriltag_Identify.getApriltagPosition(img)
            if model == 'Exit':
                capture.release()
                break
            fps.show_fps(img)
            imgbox.value = cv.imencode('.jpg', img)[1].tobytes()
        except Exception as e:
            print("program end")
            print(e)
            capture.release()

In [8]:
display(display_box,output)
threading.Thread(target=camera, ).start()

Output()

[ WARN:0@4.480] global cap_gstreamer.cpp:1777 open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1
